<a href="https://colab.research.google.com/github/luck058/kaggle-gold-price-analysis/blob/model-1/model_1_kaggle_gold_price_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/luck058/kaggle-gold-price-analysis

%cd kaggle-gold-price-analysis

Cloning into 'kaggle-gold-price-analysis'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 14), reused 13 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 667.94 KiB | 9.41 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/kaggle-gold-price-analysis


In [3]:
df = pd.read_csv('financial_regression_cleaned.csv')

display(df.head())
print(df.shape)

# Only care about sp500
df = df[['date', 'sp500 open', 'sp500 high', 'sp500 low', 'sp500 close', 'sp500 volume', 'sp500 high-low']].reset_index(drop=True)


,Unnamed: 0,date,sp500 open,sp500 high,sp500 low,sp500 close,sp500 volume,sp500 high-low,nasdaq open,nasdaq high,...,palladium high,palladium low,palladium close,palladium volume,palladium high-low,gold open,gold high,gold low,gold close,gold volume
0,0,2010-01-14,114.49,115.14,114.42,114.93,115646960.0,0.72,46.26,46.520,...,45.02,43.86,44.84,364528.0,1.16,111.51,112.37,110.79,112.03,18305238.0
1,1,2010-01-15,114.73,114.84,113.20,113.64,212252769.0,1.64,46.46,46.550,...,45.76,44.40,45.76,442210.0,1.36,111.35,112.01,110.38,110.86,18000724.0
2,3,2010-01-19,113.62,115.13,113.59,115.06,138671890.0,1.54,45.96,46.640,...,47.08,45.70,46.94,629150.0,1.38,110.95,111.75,110.83,111.52,10467927.0
3,4,2010-01-20,114.28,114.45,112.98,113.89,216330645.0,1.47,46.27,46.604,...,47.31,45.17,47.05,643198.0,2.14,109.97,110.05,108.46,108.94,17534231.0
4,5,2010-01-21,113.92,114.27,111.56,111.70,344747028.0,2.71,46.06,46.350,...,46.98,45.07,45.30,388457.0,1.91,108.48,108.78,106.61,107.37,25747831.0


(3719, 45)
